<a href="https://colab.research.google.com/github/younghoonNa/DACON_Predict_Abalone_age/blob/main/%EB%8D%B0%EC%9D%B4%EC%BD%98_%EC%A0%84%EB%B3%B5%EB%82%98%EC%9D%B4%EC%98%88%EC%B8%A1%EB%8C%80%ED%9A%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 라이브러리 설치
!pip install gdrive_dataset

In [ ]:
!pip install pycaret[full]

In [3]:
# https://drive.google.com/file/d/1RDAVsCCQCs1bxjq_2Q6qPlGeztnQ2AMD/view
from gdrivedataset import loader

file_id = "1RDAVsCCQCs1bxjq_2Q6qPlGeztnQ2AMD"
loader.load_from_google_drive(file_id)

========== files ============

data/data



In [4]:
!unzip -qq "/content//dataset.zip" -d  "/content/dataset" -y

replace /content/dataset/data/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [101]:
train = pd.read_csv('/content/dataset/data/train.csv')
train

,id,Gender,Lenght,Diameter,Height,Whole Weight,Shucked Weight,Viscra Weight,Shell Weight,Target
0,1,M,0.605,0.470,0.115,1.1140,0.3925,0.2910,0.3100,15
1,2,I,0.430,0.315,0.095,0.3780,0.1750,0.0800,0.1045,8
2,3,I,0.580,0.490,0.195,1.3165,0.5305,0.2540,0.4100,18
3,4,M,0.535,0.405,0.175,1.2705,0.5480,0.3265,0.3370,13
4,5,I,0.310,0.235,0.090,0.1270,0.0480,0.0310,0.0400,6
...,...,...,...,...,...,...,...,...,...,...
1248,1249,I,0.190,0.145,0.040,0.0380,0.0165,0.0065,0.0150,4
1249,1250,I,0.395,0.310,0.085,0.3170,0.1530,0.0505,0.0935,7
1250,1251,F,0.525,0.410,0.115,0.7745,0.4160,0.1630,0.1800,7
1251,1252,F,0.445,0.335,0.110,0.4355,0.2025,0.1095,0.1195,6


In [102]:
test = pd.read_csv('/content/dataset/data/test.csv')
test

,id,Gender,Lenght,Diameter,Height,Whole Weight,Shucked Weight,Viscra Weight,Shell Weight
0,1,F,0.595,0.470,0.155,1.1210,0.4515,0.1780,0.1550
1,2,M,0.580,0.450,0.150,0.9270,0.2760,0.1815,0.3600
2,3,I,0.260,0.205,0.070,0.0970,0.0415,0.0190,0.0305
3,4,M,0.590,0.460,0.130,1.1020,0.4550,0.2055,0.3300
4,5,F,0.595,0.465,0.140,1.1130,0.5175,0.2440,0.3050
...,...,...,...,...,...,...,...,...,...
2919,2920,I,0.170,0.105,0.035,0.0340,0.0120,0.0085,0.0050
2920,2921,I,0.435,0.345,0.115,0.4180,0.2220,0.0735,0.1060
2921,2922,I,0.570,0.450,0.135,0.7940,0.3815,0.1415,0.2450
2922,2923,I,0.460,0.350,0.120,0.4885,0.1930,0.1050,0.1550


In [103]:
sample_submission = pd.read_csv('/content/dataset/data/sample_submission.csv')
sample_submission

,id,Target
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
2919,2920,0
2920,2921,0
2921,2922,0
2922,2923,0


In [104]:
#전복 내장무게가 전복보다 많이 나가면 이상치니까.
train = train.drop(train[train["Shucked Weight"] < train["Viscra Weight"]].index[0], axis = 0)

### 편향 된거 skew로 찾아 np.log1p 취해주기

In [105]:
li = train.columns
li = ['Lenght', 'Diameter', 'Height', 'Whole Weight',
       'Shucked Weight', 'Viscra Weight', 'Shell Weight', 'Target']

for xstr in li:
    print(xstr, "Skew :", train[xstr].skew())

Lenght Skew : -0.562319641269739
Diameter Skew : -0.5231327753022951
Height Skew : -0.10938807015267188
Whole Weight Skew : 0.5798328814437481
Shucked Weight Skew : 0.7968623517244835
Viscra Weight Skew : 0.6821376675054558
Shell Weight Skew : 0.6986362872673847
Target Skew : 1.153709314313653


In [106]:
test_li = ['Lenght', 'Diameter', 'Height', 'Whole Weight',
       'Shucked Weight', 'Viscra Weight', 'Shell Weight']

for xstr in test_li:
    print(xstr, "Skew :", test[xstr].skew())

Lenght Skew : -0.6743739286022714
Diameter Skew : -0.6478774606316224
Height Skew : 4.157024003759093
Whole Weight Skew : 0.5072455586695517
Shucked Weight Skew : 0.6844586591526121
Viscra Weight Skew : 0.5487691150384756
Shell Weight Skew : 0.582689133294749


In [107]:
train['Height'] = np.log1p(train['Height'])
test['Height'] = np.log1p(test['Height'])

In [108]:
train['Target'] = np.log1p(train['Target'])
train['Target']

0       2.772589
1       2.197225
2       2.944439
3       2.639057
4       1.945910
          ...   
1248    1.609438
1249    2.079442
1250    2.079442
1251    1.945910
1252    2.484907
Name: Target, Length: 1252, dtype: float64

In [109]:
from pycaret.regression import*

setup_ = setup(data = train, target = 'Target', session_id=42,  n_jobs = -1, train_size = 0.9, silent =True) 
 # 데이터, 타겟y값, 재현을 위한 session_id 임의로 설정

,Description,Value
0,session_id,42
1,Target,Target
2,Original Data,"(1252, 10)"
3,Missing Values,False
4,Numeric Features,8
5,Categorical Features,1
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(1126, 11)"


In [110]:
import numpy as np

def NMAE(true, pred):
    mae = np.mean(np.abs(true-pred))
    score = mae / np.mean(np.abs(true))
    return score

In [111]:
add_metric('NMAE', 'NMAE', NMAE, greater_is_better = False) #Custom metric 추가

Name                                                       NMAE
Display Name                                               NMAE
Score Function                <function NMAE at 0x7f361649fcb0>
Scorer               make_scorer(NMAE, greater_is_better=False)
Target                                                     pred
Args                                                         {}
Greater is Better                                         False
Custom                                                     True
Name: NMAE, dtype: object

In [112]:
top3 = compare_models(sort='NMAE', n_select=7)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,NMAE,TT (Sec)
et,Extra Trees Regressor,0.1391,0.0340,0.1840,0.5786,0.0541,0.0593,0.0593,0.572
rf,Random Forest Regressor,0.1396,0.0335,0.1829,0.5838,0.0538,0.0594,0.0595,0.710
gbr,Gradient Boosting Regressor,0.1402,0.0345,0.1853,0.5737,0.0544,0.0596,0.0597,0.181
catboost,CatBoost Regressor,0.1411,0.0348,0.1861,0.5685,0.0548,0.0601,0.0601,2.750
lar,Least Angle Regression,0.1413,0.0344,0.1850,0.5744,0.0546,0.0602,0.0602,0.016
br,Bayesian Ridge,0.1413,0.0344,0.1850,0.5744,0.0546,0.0602,0.0602,0.014
lr,Linear Regression,0.1413,0.0344,0.1850,0.5744,0.0546,0.0602,0.0602,0.427
ridge,Ridge Regression,0.1429,0.0357,0.1885,0.5590,0.0557,0.0610,0.0609,0.015
lightgbm,Light Gradient Boosting Machine,0.1437,0.0356,0.1883,0.5586,0.0554,0.0612,0.0612,0.137
huber,Huber Regressor,0.1535,0.0428,0.2057,0.4735,0.0604,0.0647,0.0654,0.050


In [113]:
tuned_top5 = [tune_model(i,n_iter = 15) for i in top3]

,MAE,MSE,RMSE,R2,RMSLE,MAPE,NMAE
Fold,,,,,,,
0,0.1297,0.0299,0.1730,0.5895,0.0517,0.0562,0.0552
1,0.1476,0.0363,0.1904,0.4572,0.0556,0.0622,0.0630
2,0.1401,0.0336,0.1833,0.6160,0.0543,0.0607,0.0602
3,0.1298,0.0315,0.1774,0.5554,0.0525,0.0562,0.0558
4,0.1453,0.0347,0.1863,0.6235,0.0572,0.0639,0.0625
5,0.1431,0.0353,0.1879,0.5176,0.0553,0.0589,0.0600
6,0.1423,0.0358,0.1893,0.5189,0.0532,0.0570,0.0596
7,0.1306,0.0259,0.1610,0.6459,0.0477,0.0553,0.0547
8,0.1473,0.0352,0.1877,0.6515,0.0562,0.0642,0.0635


In [114]:
blender_top5 = stack_models(estimator_list=tuned_top5,optimize='NMAE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE,NMAE
Fold,,,,,,,
0,0.1321,0.0288,0.1698,0.6044,0.0507,0.0571,0.0562
1,0.1320,0.0309,0.1759,0.5370,0.0516,0.0562,0.0563
2,0.1343,0.0298,0.1727,0.6593,0.0517,0.0587,0.0577
3,0.1231,0.0300,0.1732,0.5762,0.0510,0.0532,0.0529
4,0.1429,0.0310,0.1761,0.6637,0.0536,0.0628,0.0615
5,0.1338,0.0299,0.1730,0.5914,0.0511,0.0554,0.0561
6,0.1316,0.0311,0.1762,0.5830,0.0494,0.0531,0.0551
7,0.1242,0.0247,0.1571,0.6629,0.0469,0.0529,0.0521
8,0.1370,0.0318,0.1784,0.6855,0.0523,0.0586,0.0591


In [115]:
# blender_top3 = blend_models(estimator_list=tuned_top5, optimize='NMAE') #voting regressor로 블렌딩합니다.
final_model = finalize_model(blender_top5)

In [116]:
prediction = predict_model(final_model, data=test)

In [119]:
round(np.expm1(prediction['Label']))

0       10.0
1       13.0
2        6.0
3       11.0
4       10.0
        ... 
2919     4.0
2920     8.0
2921    10.0
2922     9.0
2923    12.0
Name: Label, Length: 2924, dtype: float64

In [120]:
sample_submission['Target'] = round(np.expm1(prediction['Label']))
sample_submission

,id,Target
0,1,10.0
1,2,13.0
2,3,6.0
3,4,11.0
4,5,10.0
...,...,...
2919,2920,4.0
2920,2921,8.0
2921,2922,10.0
2922,2923,9.0


In [121]:
sample_submission.to_csv("Abalone.csv",index=False)